# Central Model Evaluation Part 3

The central model is evaluated for the cincinnati dataset.
Part 3 is used as a comparison for the flower federated learning approach.
Here, a test was executed on Raspberry Pis as physical clients.
The data was federated and trained on them.
The same data partition is to be trained on in this model with a similar model architecture.

## Imports

In [1]:
import collections
import functools
import os
import time

import numpy as np
import tensorflow as tf
import pandas as pd
import numpy as np

from tensorflow import feature_column
from tensorflow.keras import layers
from tqdm import tqdm
import import_ipynb

In [2]:
from model_helper import ModelHelper

importing Jupyter notebook from model_helper.ipynb


## DataSet

This time, only the zones used in the physical test with flowers are used for training.

In [3]:
df = pd.read_csv("./cincinnati/cincinatti_zones.csv")
df.head(10)

,location_id,vehicle_id,is_weekend,clock_sin,clock_cos,day_sin,day_cos,month_sin,month_cos,week_day_sin,week_day_cos
0,0,116,False,0.948254,-0.317512,0.994522,-0.104528,0.500000,0.866025,0.974928,-0.222521
1,1,457,False,0.902523,-0.430642,0.406737,0.913545,0.500000,0.866025,0.433884,-0.900969
2,2,153,False,0.382415,-0.923991,-0.994522,-0.104528,0.500000,0.866025,0.974928,-0.222521
3,3,215,False,0.960030,-0.279899,-0.587785,-0.809017,0.866025,0.500000,0.781831,0.623490
4,1,303,False,0.065040,-0.997883,0.743145,0.669131,0.866025,0.500000,0.781831,0.623490
5,4,107,True,0.852564,-0.522623,0.994522,-0.104528,0.866025,0.500000,-0.974928,-0.222521
6,1,383,False,0.949608,-0.313440,0.951057,0.309017,0.866025,0.500000,0.433884,-0.900969
7,5,130,False,0.894934,-0.446198,-0.951057,-0.309017,0.500000,0.866025,0.781831,0.623490
8,2,445,False,0.169350,-0.985556,-0.406737,-0.913545,0.500000,0.866025,-0.433884,-0.900969
9,4,457,False,0.533492,-0.845805,-0.207912,0.978148,0.500000,0.866025,0.974928,-0.222521


In [4]:
# the number of different locations defines the vocabulary size
locations = df.location_id
vocab_size = locations.nunique()

print('vocabulary size:', vocab_size)

vocabulary size: 100


Init the ModelHelper and set all needed parameters such as the different column_names and the vocab_size.

In [5]:
mh = ModelHelper(df, 17)

The dataset is limited to only include two different client_ids.
This way the results between this centralized model and the model trained by flower are comparable.
It was only managed to get flower working on two Raspberry PIs, therefor only two clients are chosen.
This is expected to significantly impact the performance of the models.

In [6]:
mh.set_target_column_name('location_id')
mh.set_vocab_size(vocab_size)

numerical_column_names = ['clock_sin', 'clock_cos', 'day_sin', 'day_cos', 'month_sin', 'month_cos', 'week_day_sin', 'week_day_cos']
column_names = ['location_id'] + numerical_column_names
mh.set_column_names(column_names)
mh.set_numerical_column_names(numerical_column_names)

mh.set_client_column_name('vehicle_id')
CLIENT_IDS = [457, 461]
mh.set_client_column_ids(CLIENT_IDS)

In [7]:
count = df.vehicle_id.value_counts()

idx = count.loc[count.index[:100]].index # count >= 100
df = df.loc[df.vehicle_id.isin(idx)]

An array is created containing all visited locations for every user.
The original data is sorted by time (ascending).
Thus, the array contains a sequence of visited locations by user.

In [8]:
mh.create_users_locations_from_df()

100%|██████████| 2/2 [00:00<00:00, 11.99it/s]


[          location_id  vehicle_id  is_weekend  clock_sin  clock_cos   day_sin  \
 1                   1         457       False   0.902523  -0.430642  0.406737   
 9                   4         457       False   0.533492  -0.845805 -0.207912   
 123                 4         457       False   0.796002  -0.605294  0.866025   
 124                 2         457       False   0.678534  -0.734569 -0.587785   
 143                 1         457       False  -0.492550  -0.870284  0.207912   
 ...               ...         ...         ...        ...        ...       ...   
 19073749            2         457       False  -0.469600  -0.882879  0.406737   
 19073823            1         457       False  -0.030611  -0.999531  0.866025   
 19074051            6         457       False  -0.485128  -0.874443 -0.743145   
 19074159            2         457       False  -0.241922  -0.970296 -0.207912   
 19074203            1         457       False  -0.030902  -0.999522  0.951057   
 
            da

It is necessary to first split the data in train, valid and test for each user.
Then, these are merged together again later on.
This is done to ensure that the sequences are kept together and not split randomly for the users.

In [9]:
mh.concat_split_users_locations()

In [10]:
print(len(mh.df_train), 'train examples')
print(len(mh.df_val), 'validation examples')
print(len(mh.df_test), 'test examples')

441368 train examples
110343 validation examples
137929 test examples


Split the data and create the batch datasets.

In [11]:
#mh.split_data_sliding()
mh.split_data()
print(len(mh.list_test))
mh.list_test[0]

8114


,location_id,is_weekend,clock_sin,clock_cos,day_sin,day_cos,month_sin,month_cos,week_day_sin,week_day_cos
13247969,1,False,0.850237,-0.526399,4.067366e-01,0.913545,-1.000000,-1.836970e-16,0.974928,-0.222521
13248042,25,False,0.156147,-0.987734,2.079117e-01,-0.978148,-0.500000,-8.660254e-01,0.781831,0.623490
13248048,1,False,0.870356,-0.492424,-1.133108e-15,1.000000,-0.500000,-8.660254e-01,0.433884,-0.900969
13248053,2,False,0.697738,-0.716353,4.067366e-01,-0.913545,-0.500000,-8.660254e-01,0.000000,1.000000
13248067,4,False,0.138813,-0.990319,9.510565e-01,-0.309017,-0.500000,-8.660254e-01,0.433884,-0.900969
13248098,2,False,0.245801,-0.969320,-4.067366e-01,-0.913545,-1.000000,-1.836970e-16,0.433884,-0.900969
13248114,29,False,-0.526276,-0.850314,-1.133108e-15,1.000000,-1.000000,-1.836970e-16,0.974928,-0.222521
13248136,7,False,-0.263242,-0.964730,-8.660254e-01,-0.500000,-0.866025,-5.000000e-01,0.433884,-0.900969
13248265,25,False,-0.386711,-0.922201,-4.067366e-01,-0.913545,-0.866025,-5.000000e-01,0.000000,1.000000
13248274,10,False,0.349935,-0.936774,9.510565e-01,0.309017,-0.500000,-8.660254e-01,0.000000,1.000000


In [12]:
mh.set_batch_size(16)
mh.create_and_batch_datasets(multi_target=False)

In [13]:
# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units1 = 256
rnn_units2 = 128

# Create a model
def create_keras_model():
  N = mh.total_window_length
  batch_size = mh.batch_size
  number_of_places = mh.vocab_size

  # Shortcut to the layers package
  l = tf.keras.layers

  # List of numeric feature columns to pass to the DenseLayer
  numeric_feature_columns = []

  # Handling numerical columns
  for header in numerical_column_names:
	# Append all the numerical columns defined into the list
    numeric_feature_columns.append(feature_column.numeric_column(header, shape=N-1))

  feature_inputs={}
  for c_name in numerical_column_names:
    feature_inputs[c_name] = tf.keras.Input((N-1,), batch_size=batch_size, name=c_name)

  # We cannot use an array of features as always because we have sequences
  # We have to do one by one in order to match the shape
  num_features = []
  for c_name in numerical_column_names:
    f =  feature_column.numeric_column(c_name, shape=(N-1))
    feature = l.DenseFeatures(f)(feature_inputs)
    feature = tf.expand_dims(feature, -1)
    num_features.append(feature)

  # Declare the dictionary for the locations sequence as before
  sequence_input = {
      'location_id': tf.keras.Input((N-1,), batch_size=batch_size, dtype=tf.dtypes.int32, name='location_id') # add batch_size=batch_size in case of stateful GRU
  }

  # Handling the categorical feature sequence using one-hot
  location_one_hot = feature_column.sequence_categorical_column_with_vocabulary_list(
      'location_id', [i for i in range(number_of_places)])

  # one-hot encoding
  location_feature = feature_column.embedding_column(location_one_hot, embedding_dim)

  # With an input sequence we can't use the DenseFeature layer, we need to use the SequenceFeatures
  sequence_features, sequence_length = tf.keras.experimental.SequenceFeatures(location_feature)(sequence_input)


  input_sequence = l.Concatenate(axis=2)([sequence_features] + num_features)

  # Rnn
  recurrent = l.GRU(rnn_units1,
                    batch_size=batch_size, #in case of stateful
                    return_sequences=True,
                    stateful=True,
                    recurrent_initializer='glorot_uniform')(input_sequence)

  recurrent_2 = l.GRU(rnn_units2,
                      batch_size=batch_size, #in case of stateful
                      stateful=True,
                      recurrent_initializer='glorot_uniform')(recurrent)


  # Softmax output layer
  # Last layer with an output for each places
  output = layers.Dense(number_of_places, activation='softmax')(recurrent_2)


  # To return the Model, we need to define its inputs and outputs
  # In out case, we need to list all the input layers we have defined
  inputs = list(feature_inputs.values()) + list(sequence_input.values())

  # Return the Model
  return tf.keras.Model(inputs=inputs, outputs=output)

In [14]:
# Get the model and compile it
mh.assign_model(create_keras_model())
mh.compile_model()

In [18]:
mh.model.summary()

TypeError: summary() got an unexpected keyword argument 'with_early_stopping'

In [19]:
mh.set_num_epochs(15)
mh.fit_model(with_early_stopping=False)

Epoch 1/15
1622/1622 [==============================] - 21s 13ms/step - loss: 2.4798 - sparse_categorical_accuracy: 0.2673 - val_loss: 2.9824 - val_sparse_categorical_accuracy: 0.1395
Epoch 2/15
1622/1622 [==============================] - 21s 13ms/step - loss: 2.4766 - sparse_categorical_accuracy: 0.2681 - val_loss: 2.9802 - val_sparse_categorical_accuracy: 0.1395
Epoch 3/15
1622/1622 [==============================] - 20s 13ms/step - loss: 2.4721 - sparse_categorical_accuracy: 0.2702 - val_loss: 2.9894 - val_sparse_categorical_accuracy: 0.1395
Epoch 4/15
1622/1622 [==============================] - 21s 13ms/step - loss: 2.4651 - sparse_categorical_accuracy: 0.2721 - val_loss: 2.9992 - val_sparse_categorical_accuracy: 0.1395
Epoch 5/15
1622/1622 [==============================] - 21s 13ms/step - loss: 2.4591 - sparse_categorical_accuracy: 0.2755 - val_loss: 2.9870 - val_sparse_categorical_accuracy: 0.1395
Epoch 6/15
1622/1622 [==============================] - 20s 13ms/step - loss: 2.

In [20]:
mh.evaluate_model()

507/507 [==============================] - 2s 4ms/step - loss: 3.3152 - sparse_categorical_accuracy: 0.1288


As expected the accuracy of the model is rather low.
But more importantly is how this centralized model compares to the federally trained model.
After 15 rounds on flower, the server reported an accuracy of approximately 0.1248.
This centralized model achieved 0.1288 with the same amount of epochs.
Thus, it can be concluded that the federal approach does not lose much accuracy compared to a centralized approach.
The main difference is going to be how long it takes.
Flower took roughly 55min to finish 15 rounds of training with two Raspberry PI 4s.
The centralized model took around 5min.
It is difficult to predict the time difference when larger sets of data and more federated clients are used.